<a href="https://colab.research.google.com/github/AI4GoodProject/speechRecForMeeting/blob/master/CNNModelWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2,glob
import random
from sklearn.model_selection import train_test_split

0.Data retrieving/some preprocessing


In [16]:
# Load the Drive helper and mount
from google.colab import drive
import pickle

# This will prompt for authorization.
drive.mount('/content/drive')

DATA_PATH = "/content/drive/My Drive/Team 6/processed-data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls "/content/drive/My Drive/Team 6/processed-data"

dialogue-acts.pkl  features.pkl


In [78]:
features= open(DATA_PATH+'/features.pkl','rb')
#meeting_df= open(DATA_PATH+'/dialogue-acts.pkl','rb')
features_list = pickle.load(features)
booleans_list = []

for i in range(0,len(features_list)):
  random_bits = random.getrandbits(5)
  booleans_list.append(bool(random_bits))


print("features_list","\n", features_list[:5])
print("length", len(features_list)) 
print("width", len(features_list[0]))
print("-----------------")
print("booleans_list","\n",booleans_list[:5])
print("length", len(booleans_list)) 
print("width", booleans_list[0])


features_list 
 [array([[7.1597932e-04, 1.9338253e-04, 4.6453884e-04, ..., 3.4911142e-04,
        8.6215977e-04, 2.1067560e-03],
       [1.8004803e-03, 6.4909243e-04, 1.0058059e-03, ..., 7.8662910e-04,
        3.6218630e-03, 1.6237851e-02],
       [4.2893961e-03, 3.8882708e-03, 1.0424453e-02, ..., 2.0834696e-03,
        2.7668604e-03, 6.4639011e-03],
       ...,
       [2.3597777e-07, 3.0304275e-07, 2.6253872e-07, ..., 3.8473129e-07,
        2.9374269e-07, 4.3489683e-07],
       [2.1526543e-07, 1.9735974e-07, 1.6411558e-07, ..., 3.0774763e-07,
        2.5692663e-07, 4.1122118e-07],
       [2.1816778e-07, 2.1505960e-07, 1.9730011e-07, ..., 2.9854834e-07,
        3.5006087e-07, 2.3322755e-07]], dtype=float32), array([[3.5758321e-03, 1.6352083e-03, 2.0024246e-03, ..., 2.8073965e-04,
        6.8266573e-04, 3.0436167e-03],
       [6.5945424e-03, 2.4174996e-02, 2.1251529e-02, ..., 5.5366373e-03,
        7.6450505e-03, 1.6457111e-02],
       [3.3398282e-02, 5.5603825e-02, 1.9763701e-02, ..., 

In [3]:
def crossJoin (list1,list2):
  crossJoined_list = []
  for i in range(0,len(list1)):
    crossJoined_list.append(list1[i])
    crossJoined_list.append(list2[i])
  return crossJoined_list

In [2]:
# for testing purpose

list1 = np.array([['a', 'b'], ['c','d'],['e','f']])
list2 = np.array([[True], [False], [True]])

In [4]:
crossJoin (list1,list2)

[array(['a', 'b'], dtype='<U1'),
 array([ True]),
 array(['c', 'd'], dtype='<U1'),
 array([False]),
 array(['e', 'f'], dtype='<U1'),
 array([ True])]

In [86]:
for image in features_list:
  print(image.shape)
  break



(128, 313)


In [79]:
print(features_list[0])

[[7.1597932e-04 1.9338253e-04 4.6453884e-04 ... 3.4911142e-04
  8.6215977e-04 2.1067560e-03]
 [1.8004803e-03 6.4909243e-04 1.0058059e-03 ... 7.8662910e-04
  3.6218630e-03 1.6237851e-02]
 [4.2893961e-03 3.8882708e-03 1.0424453e-02 ... 2.0834696e-03
  2.7668604e-03 6.4639011e-03]
 ...
 [2.3597777e-07 3.0304275e-07 2.6253872e-07 ... 3.8473129e-07
  2.9374269e-07 4.3489683e-07]
 [2.1526543e-07 1.9735974e-07 1.6411558e-07 ... 3.0774763e-07
  2.5692663e-07 4.1122118e-07]
 [2.1816778e-07 2.1505960e-07 1.9730011e-07 ... 2.9854834e-07
  3.5006087e-07 2.3322755e-07]]


1. Train/validate/test split

In [97]:
# Manually set up training/validation/testing dataset
def split_three(list, train_r, val_r, test_r ):
    random.shuffle(list)
    c1 = int(len(list)*train_r)
    c2 = int(len(list)* (train_r + val_r))
    
    train = list[:c1]
    val = list[c1:c2]
    test = list[c2:]

    return train, val, test

In [100]:
X_train, X_val, X_test = split_three(features_list,0.6, 0.2, 0.2)
Y_train, Y_val, Y_test = split_three(booleans_list,0.6, 0.2, 0.2)

In [105]:
trainloader = crossJoin(X_train, Y_train)
valloader = crossJoin(X_val, Y_val)
testloader = crossJoin(X_test,Y_test)

In [106]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [110]:
#evaluation
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [111]:
#train model
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
#[[X_train_1, y_train_1]...[[X_train_{}, y_train_{}]]
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        data = torch.tensor(data)
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

ValueError: ignored